# Análisis y Limpieza de Datos de Ventas

## Contexto

Tienes un conjunto de datos que representa las ventas diarias de una tienda en línea. Este conjunto contiene información sobre los productos vendidos, la cantidad, el precio por unidad, y la fecha de venta. Sin embargo, algunos de los datos están incompletos o tienen valores nulos. El objetivo de este ejercicio es limpiar y analizar los datos, y calcular algunas medidas clave para entender mejor las ventas.

## Objetivos

* Manipular arrays con numpy: Para operaciones matemáticas básicas con los datos.
* Manejar datos con pandas: Para cargar el dataset, analizar su estructura y calcular medidas de tendencia central.
* Trabajar con archivos JSON: Para guardar y leer la información transformada.
* Manejar datos nulos: Para identificar y lidiar con valores faltantes.
* Aplicar ciclos, condicionales y operadores lógicos/matemáticos: Para realizar cálculos y aplicar reglas de negocio.
